# GATK Tutorial | Hard Filtering | March 2019


This GATK tutorial corresponds to a section of the GATK Workshop _2b. Germline Hard Filtering Tutorial_ worksheet. The goal is to become familiar with germline variant annotations. The notebook illustrates the following steps. 

- Use GATK to stratify a variant callset against a truthset
- Use R's ggplot2 package to plot the distribution of various annotation values
- Hard-filter based on annotation thresholds and calculate concordance metrics  

### First, make sure the notebook is using a Python 3 kernel in the top right corner.
A kernel is a _computational engine_ that executes the code in the notebook. We use Python 3 in this notebook to execute GATK commands using _Python Magic_ (`!`). Later we will switch to another notebook to do some plotting in R.

### How to run this notebook:
- **Click to select a gray cell and then pressing SHIFT+ENTER to run the cell.**

- **Write results to `/home/jupyter-user/`. To access the directory, click on the upper-left jupyter icon.**

### Enable reading Google bucket data 

In [ ]:
# Check if data is accessible. The command should list several gs:// URLs.
! gsutil ls gs://gatk-tutorials/workshop_1702/variant_discovery/data/resources/
! gsutil ls gs://gatk-tutorials/workshop_1702/variant_discovery/data/intervals/motherHighconf.bed
! gsutil ls gs://gatk-tutorials/workshop_1702/variant_discovery/data/inputVcfs/

In [ ]:
# If you do not see gs:// URLs listed above, run this cell to install Google Cloud Storage. 
# Afterwards, restart the kernel with Kernel > Restart.
#! pip install google-cloud-storage

---
## 1. Subset variants to SNPs of a single sample with SelectVariants

Subset the trio callset to just the SNPs of the mother (sample NA12878). Make sure to remove sites for which the sample genotype is homozygous-reference and remove unused alleles, including spanning deletions. 

> The tool recalculates depth of coverage (DP) per site as well as the allele count in genotypes for each ALT allele (AC), allele frequency for each ALT allele (AF), and  total number of alleles in called genotypes (AN), to reflect only the subset sample(s).

In [1]:
! gatk SelectVariants \
-V gs://gatk-tutorials/workshop_1702/variant_discovery/data/inputVcfs/trio.vcf.gz \
-sn NA12878 \
-select-type SNP \
--exclude-non-variants \
--remove-unused-alternates \
-O /home/jupyter-user/motherSNP.vcf.gz

Using GATK jar /etc/gatk-4.1.0.0/gatk-package-4.1.0.0-local.jar
Running:
    java -Dsamjdk.use_async_io_read_samtools=false -Dsamjdk.use_async_io_write_samtools=true -Dsamjdk.use_async_io_write_tribble=false -Dsamjdk.compression_level=2 -jar /etc/gatk-4.1.0.0/gatk-package-4.1.0.0-local.jar SelectVariants -V gs://gatk-tutorials/workshop_1702/variant_discovery/data/inputVcfs/trio.vcf.gz -sn NA12878 -select-type SNP --exclude-non-variants --remove-unused-alternates -O /home/jupyter-user/motherSNP.vcf.gz
05:34:16.167 INFO  NativeLibraryLoader - Loading libgkl_compression.so from jar:file:/etc/gatk-4.1.0.0/gatk-package-4.1.0.0-local.jar!/com/intel/gkl/native/libgkl_compression.so
05:34:16.395 INFO  SelectVariants - ------------------------------------------------------------
05:34:16.396 INFO  SelectVariants - The Genome Analysis Toolkit (GATK) v4.1.0.0
05:34:16.396 INFO  SelectVariants - For support and documentation go to https://software.broadinstitute.org/gatk/
05:34:16.396 INFO  Select

In [2]:
# Peruse the resulting file 
! zcat /home/jupyter-user/motherSNP.vcf.gz | grep -v '##' | head

#CHROM	POS	ID	REF	ALT	QUAL	FILTER	INFO	FORMAT	NA12878
20	61098	.	C	T	465.13	.	AC=1;AF=0.500;AN=2;BaseQRankSum=0.516;ClippingRankSum=0.00;DP=44;ExcessHet=3.0103;FS=0.000;MQ=59.48;MQRankSum=0.803;QD=10.57;ReadPosRankSum=1.54;SOR=0.603	GT:AD:DP:GQ:PL	0/1:28,16:44:99:496,0,938
20	61795	.	G	T	2034.16	.	AC=1;AF=0.500;AN=2;BaseQRankSum=-6.330e-01;ClippingRankSum=0.00;DP=60;ExcessHet=3.9794;FS=0.000;MQ=59.81;MQRankSum=0.00;QD=17.09;ReadPosRankSum=1.23;SOR=0.723	GT:AD:DP:GQ:PL	0/1:30,30:60:99:1003,0,1027
20	63244	.	A	C	923.13	.	AC=1;AF=0.500;AN=2;BaseQRankSum=0.637;ClippingRankSum=0.00;DP=57;ExcessHet=3.0103;FS=5.470;MQ=59.60;MQRankSum=-1.019e+00;QD=16.20;ReadPosRankSum=0.404;SOR=1.528	GT:AD:DP:GQ:PL	0/1:30,27:57:99:954,0,1064
20	63799	.	C	T	1766.16	.	AC=1;AF=0.500;AN=2;BaseQRankSum=-6.530e-01;ClippingRankSum=0.00;DP=45;ExcessHet=3.9794;FS=0.000;MQ=59.78;MQRankSum=1.22;QD=16.98;ReadPosRankSum=-1.075e+00;SOR=0.709	GT:AD:DP:GQ:PL	0/1:19,26:45:99:953,0,670
20	65900	.	G	A	5817.13	.	AC=1;AF=0.500;AN

---
## 2. Annotate intersecting true positives with VariantAnnotator

We use VariantAnnotator to annotate which variants in our callset are also present in the truthset (GIAB), which are considered true positives. Variants not present in the truthset are considered false positives. Here we produce a callset where variants that are present in the truthset are annotated with the giab.callsets annotation plus a value indicating how many of the callsets used to develop the truthset agreed with that call.

In [ ]:
! gatk VariantAnnotator \
-V /home/jupyter-user/motherSNP.vcf.gz \
--resource:giab gs://gatk-tutorials/workshop_1702/variant_discovery/data/resources/motherGIABsnps.vcf.gz \
-E giab.callsets \
-O /home/jupyter-user/motherSNP.giab.vcf.gz

In [ ]:
# Peruse the resulting file 
! zcat /home/jupyter-user/motherSNP.giab.vcf.gz | grep -v '##' | head

---
## 3. Tabulate annotations of interest with VariantsToTable

Convert the information from the callset into a tab delimited table using VariantsToTable, so that we can parse it easily in R. The tool parameters differentiate INFO/site-level fields fields (`-F`) and FORMAT/sample-level fields genotype fields (`-GF`). This step produces a table where each line represents a variant record from the VCF, and each column represents an annotation we have specified. Wherever the requested annotations are not present, e.g. RankSum annotations at homozygous sites, the value will be replaced by NA. 

In [ ]:
! gatk VariantsToTable \
-V /home/jupyter-user/motherSNP.giab.vcf.gz \
-F CHROM -F POS -F QUAL \
-F BaseQRankSum -F MQRankSum -F ReadPosRankSum \
-F DP -F FS -F MQ -F QD -F SOR \
-F giab.callsets \
-GF GQ \
-O /home/jupyter-user/motherSNP.giab.txt

In [ ]:
# Peruse the resulting file
! cat /home/jupyter-user/motherSNP.giab.txt | head -n300

In [ ]:
# Focus in on a few columns
! cat /home/jupyter-user/motherSNP.giab.txt | cut -f1,2,7,12 | head -n300


---
## 4. Make density and scatter plots in R and determine filtering thresholds

<span style="color:red">Load the R notebook now to run the plots for this next section. Continue below only after you've finished with the other notebook.</span>


---
## 5. Apply filters with VariantFiltration and evaluate results

### A. Filter on QUAL and tabulate baseline concordance

Based on the plots we generated, we're going to apply some filters to weed out false positives. To illustrate how VariantFiltration works, and to establish baseline performance, we first filter on QUAL < 30. By default, GATK GenotypeGVCFs filters out variants with QUAL < 10. This step produces a VCF with all the original variants; those that failed the filter are annotated with the filter name in the FILTER column.


In [ ]:
# Filter callset on one annotation, QUAL < 30
! gatk VariantFiltration \
-R gs://gatk-tutorials/workshop_1702/variant_discovery/data/ref/ref.fasta \
-V /home/jupyter-user/motherSNP.vcf.gz \
--filter-expression "QUAL < 30" \
--filter-name "qual30" \
-O /home/jupyter-user/motherSNPqual30.vcf.gz

In [ ]:
# Peruse the results; try adding 'grep "qual30"'
! zcat /home/jupyter-user/motherSNPqual30.vcf.gz | grep -v '##' | head -n10

In [ ]:
# Calculate concordance metrics using GATK4 BETA tool Concordance
! gatk Concordance \
-eval /home/jupyter-user/motherSNPqual30.vcf.gz \
-truth gs://gatk-tutorials/workshop_1702/variant_discovery/data/resources/motherGIABsnps.vcf.gz \
-L gs://gatk-tutorials/workshop_1702/variant_discovery/data/intervals/motherHighconf.bed \
-S /home/jupyter-user/motherSNPqual30.txt

In [ ]:
# View the results
! echo ""
! cat /home/jupyter-user/motherSNPqual30.txt

### B. Filter on multiple annotations simultaneously using VariantFiltration

To filter on multiple expressions, provide each in separate expression. For INFO level annotations, the parameter is  `-filter`, which should be immediately followed by the corresponding `–-filter-name` label. Here we show basic hard-filtering thresholds.

- If an annotation is missing, VariantFiltration skips any judgement on that annotation. To conservatively fail such missing annotation sites, set the `--missing-values-evaluate-as-failing` flag. 
- To filter based on FORMAT level annotations, use `--genotype-filter-expression` and `--genotype-filter-name`. 

In [ ]:
# Filter callset on multiple annotations.
# Iterate on thresholds to improve precision while maintaining high sensitivity.
! gatk VariantFiltration \
-V /home/jupyter-user/motherSNP.vcf.gz \
-filter "QD < 2.0" --filter-name "QD2" \
-filter "QUAL < 30.0" --filter-name "QUAL30" \
-filter "SOR > 3.0" --filter-name "SOR3" \
-filter "FS > 60.0" --filter-name "FS60" \
-filter "MQ < 40.0" --filter-name "MQ40" \
-filter "MQRankSum < -12.5" --filter-name "MQRankSum-12.5" \
-filter "ReadPosRankSum < -8.0" --filter-name "ReadPosRankSum-8" \
-O /home/jupyter-user/motherSNPfilters.vcf.gz

In [ ]:
# Sanity-check that filtering is as expected by examining filtered records and PASS records.
! zcat /home/jupyter-user/motherSNPfilters.vcf.gz | grep -v '##' | grep -v 'PASS' | head -n20 | cut -f6-10
! zcat /home/jupyter-user/motherSNPfilters.vcf.gz | grep -v '#' | grep 'PASS' | head | cut -f6-10

In [ ]:
# Calculate concordance metrics using GATK4 BETA tool Concordance
! gatk Concordance \
-eval /home/jupyter-user/motherSNPfilters.vcf.gz \
-truth gs://gatk-tutorials/workshop_1702/variant_discovery/data/resources/motherGIABsnps.vcf.gz \
-L gs://gatk-tutorials/workshop_1702/variant_discovery/data/intervals/motherHighconf.bed \
-S /home/jupyter-user/motherSNPfilters.txt
        

In [ ]:
#Now lets re-run concordance from just using QUAL filtering first
!cat /home/jupyter-user/motherSNPqual30.txt

In [ ]:
# View the results from filtering on multiple annotations
! echo ""
! cat /home/jupyter-user/motherSNPfilters.txt

---

We performed hard-filtering to learn about germline variant annotations. Remember that GATK recommends _Variant Quality Score Recalibration_ (VQSR) for germline variant callset filtering. For more complex variant filtering and annotation, see the Broad [Hail.is](https://hail.is/index.html) framework. 